In [2]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from scipy.spatial.distance import cosine
import pickle

def preprocess_image(image_path, input_size=(224, 224)):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = image.resize(input_size)
    image_array = np.array(image)
    image_array = image_array / 255.0
    image_array = image_array.astype(np.float32)
    image_array = np.transpose(image_array[np.newaxis, ...], (0,3,1,2))
    return image_array

def run_inference(image_path, model_path):
    session = ort.InferenceSession(model_path)
    input_name = session.get_inputs()[0].name
    input_data = preprocess_image(image_path)
    outputs = session.run(None, {input_name: input_data})
    probabilities = softmax(outputs[0][0])
    predicted_class = np.argmax(probabilities)
    return predicted_class, probabilities

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [3]:
import os

# 이미지가 있는 기본 디렉토리 경로
base_dir = "/workspace/a-eye-lab-research/dataset/real_data"
model_path = "/workspace/model_quantized.onnx"

# 모든 하위 디렉토리(0, 1)를 순회
for class_dir in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_dir)
    if os.path.isdir(class_path):
        print(f"\nProcessing class directory: {class_dir}")
        
        # 각 클래스 디렉토리 내의 모든 이미지 파일을 순회
        for image_file in os.listdir(class_path):
            if image_file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(class_path, image_file)
                
                # 추론 실행
                predicted_class,probabilities = run_inference(image_path, model_path)

                # 결과 출력
                print(f"\nImage: {image_file}")
                print(f"True class: {class_dir}")
                print(f"Predicted class: {predicted_class}")
                print(f"Probabilities: [{probabilities[0]:.4f}, {probabilities[1]:.4f}]")


Processing class directory: 0

Image: 20_20250207_left.jpeg
True class: 0
Predicted class: 0
Probabilities: [0.8955, 0.1045]

Image: 30_20250207_right.jpeg
True class: 0
Predicted class: 0
Probabilities: [0.5500, 0.4500]

Image: 23_20250207_right.jpeg
True class: 0
Predicted class: 0
Probabilities: [0.9317, 0.0683]

Image: 08_20250207_left.jpg
True class: 0
Predicted class: 0
Probabilities: [0.7871, 0.2129]

Image: 35_20250207_right.jpeg
True class: 0
Predicted class: 0
Probabilities: [0.7782, 0.2218]


KeyboardInterrupt: 